In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from lib.explore import print_df
from lib.match import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19

In [2]:
pd_cprr = clean_18()
pd_cprr

standardize_from_lookup_table: unmatched sequences:
  {'', ', ', '; ', ' & ', ' (60 day rule)', 'command of temper (', ': ', 'awol '}


,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,rule_code,...,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,agency,data_production_year
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,,3:17,...,2018,1,2,2017,12,30,2018-002,4990106e2df85caab5910fe45de338ef,Baton Rouge PD,2017
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,,3:17,...,2018,1,9,2018,1,9,2018-004,ddbd2be4c2f4c79dfd5aaa4b4e49212d,Baton Rouge PD,2018
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,,2:4,...,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,Baton Rouge PD,2017
3,office investigation,office investigation,office investigation,P10476,op service comm center,rimes,melissa,j,,2:4,...,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,Baton Rouge PD,2017
4,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,,3:20,...,2018,1,10,2018,1,10,2018-005,9d1a70ded8a2660295b9a4ad9cb357a5,Baton Rouge PD,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,3:17,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
145,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,,3:18,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
146,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2:12,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
147,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,,3:17,...,2018,1,26,2018,1,26,2018-010,adcf8ba6410ae8beb2311493fb0a8f29,Baton Rouge PD,2018


## Matching 2018 Baton Rouge PD CPRR against 2017 Baton Rouge CSD PPRR

In [3]:
csd_17 = clean_csd_17()

csd_of = csd_17[["first_name", "last_name", "middle_initial", "employee_id"]].drop_duplicates("employee_id").set_index("employee_id", drop=True)
csd_of.loc[:, "fc"] = csd_of.first_name.map(lambda x: x[:1])

pd_of = pd_cprr[["first_name", "last_name", "middle_initial", "uid"]].drop_duplicates("uid").set_index("uid", drop=True)
pd_of.loc[:, "fc"] = pd_of.first_name.fillna("").map(lambda x: x[:1])

matcher = ThresholdMatcher(pd_of, csd_of, ColumnsIndex(["fc"]), {
    "last_name": JaroWinklerSimilarity(),
    "first_name": JaroWinklerSimilarity(),
    "middle_initial": JaroWinklerSimilarity(),
})

In [4]:
print_df(matcher.get_sample_pairs(step=0.025, lower_bound=0.9))
matcher.print_decision(0.98)

                                                                  first_name  last_name middle_initial fc
score_range pair_idx sim_score row_key                                                                   
1.00-0.97   0        1.000000  5021017be642864ea72ade38867279b5  christopher     taylor              b  c
                               329150                            christopher     taylor              b  c
            1        1.000000  aad73c46beb5ce5de2e6a864433199d5        chase        ard              r  c
                               440671                                  chase        ard              r  c
            2        1.000000  8012c7dc3c5f5de7f2eae3850c8e7c95      charles  pollar jr              g  c
                               456764                                charles  pollar jr              g  c
            3        0.989014  2b4231ab5f80219af8b660770991edbf     jonathan     migues              j  j
                               457167         

## Matching 2018 Baton Rouge PD CPRR against 2019 Baton Rouge CSD PPRR

In [5]:
csd_19 = clean_csd_19()

csd_of = csd_19[["first_name", "last_name", "middle_initial", "employee_id"]].drop_duplicates("employee_id").set_index("employee_id", drop=True)
csd_of.loc[:, "fc"] = csd_of.first_name.map(lambda x: x[:1])

pd_of = pd_cprr[["first_name", "last_name", "middle_initial", "uid"]].drop_duplicates("uid").set_index("uid", drop=True)
pd_of.loc[:, "fc"] = pd_of.first_name.fillna("").map(lambda x: x[:1])

matcher = ThresholdMatcher(pd_of, csd_of, ColumnsIndex(["fc"]), {
    "last_name": JaroWinklerSimilarity(),
    "first_name": JaroWinklerSimilarity(),
    "middle_initial": JaroWinklerSimilarity(),
})

In [6]:
print_df(matcher.get_sample_pairs(step=0.025, lower_bound=0.9))
matcher.print_decision(0.96)

                                                                          first_name  last_name middle_initial fc
score_range pair_idx sim_score row_key                                                                           
1.00-0.97   0        0.985408  dcd668b2249d9719f134d9bf1c1e943e               donald  steele jr                 d
                               0x8980DC86C1E9EA324EC99F8B15A2E9A92D0A4D1E     donald     steele                 d
            1        0.985408  4acbef036449bb9387ec486879bf2e31               ronald  norman jr              w  r
                               0x73EE4958BDB5A056029EBD39B8ABBAA3DBC0F333     ronald     norman              w  r
            2        0.985408  8012c7dc3c5f5de7f2eae3850c8e7c95              charles  pollar jr              g  c
                               0x4C1B52409CF6BE3896CF163FA17B32E4DA293F2E    charles     pollar              g  c
            3        0.979610  089f6dd12d3f16db38b712992e4bcb32               donald   y